In [59]:
import nltk 
import os
from dash import Dash,dcc,Output,Input
import dash_bootstrap_components as dbc
import dash_html_components as html
import plotly.graph_objects as go
import selenium
import requests
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from unidecode import unidecode
from nltk import pos_tag
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
import nltk, re, string, collections
from nltk.stem import WordNetLemmatizer
import pandas as pd
from nltk.util import ngrams
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.collocations import *
from nltk.collocations import BigramCollocationFinder,TrigramCollocationFinder,QuadgramCollocationFinder
from hazm import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mahnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mahnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mahnaz\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
path='chromedriver.exe'
driver=webdriver.Chrome(executable_path=path)
driver.get('https://www.zoomit.ir/archive/?sort=Newest&groupings=33213')
time.sleep(5)
driver.execute_script("window.scrollTo(0,2000)")
time.sleep(2)
driver.execute_script("window.scrollTo(2000,4000)")
time.sleep(2)
bishtar_bar=driver.find_elements(By.CSS_SELECTOR,'.flex__Flex-le1v16-0.eEklvK')

for i,j in enumerate(bishtar_bar):
    if i>7:
        j.click()

time.sleep(3)
driver.execute_script("window.scrollTo(4000,6000)")
time.sleep(2)
driver.execute_script("window.scrollTo(6000,8000)")
time.sleep(2)
bishtar_bar1=driver.find_element(By.XPATH,'//*[@id="__next"]/div[2]/div[1]/div[3]/div/div/div[2]/div/div[3]/button/div')
bishtar_bar1.click()
time.sleep(3)
driver.execute_script("window.scrollTo(8000,10000)")
time.sleep(2)
driver.execute_script("window.scrollTo(10000,12000)")
time.sleep(2)

matn=''
div_bar=driver.find_element(By.CSS_SELECTOR,'.flex__Flex-le1v16-0.cVZflE')
class_bars=div_bar.find_elements(By.CSS_SELECTOR,'.link__CustomNextLink-sc-1r7l32j-0.iCQspp')

for i in class_bars:
    h=i.get_attribute('href')
    matn+=str(h)+'\n'
    with open('links.txt' , 'w' ,encoding='utf-8') as ss:
        ss.write(matn)


In [ ]:
with open('links.txt' , 'r' ,encoding='utf-8') as ss:
    link=ss.read()

link=link.split()

matn_koli=[]
matn1=''
for j,i in enumerate(link):
    driver.get(i)
    time.sleep(3)
    driver.execute_script("window.scrollTo(0,2000)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(2000,4000)")
    time.sleep(4)
    matn_bar=driver.find_elements(By.TAG_NAME,'p')
    for o in matn_bar:
        m=o.get_attribute('innerHTML')
        matn1+=str(m)+'\n'
            
    with open('matn.txt' , 'w' ,encoding='utf-8') as ss:
            ss.write(matn1)


In [78]:

def splitt(matn):
    global text
    f = open(matn, "r",encoding='utf-8')
    text = f.read()
    text=text.split('\n\n')
    for i in text:
        if i=='':
            text.remove(i)
    return text

def preprocess(textt,stop_word):

    global normal_text
    global sents_text
    global words_text
    global words_without_stopwords
    global words_tag
    global lemma_words
    global fixd_lemma_words
    global fixdd_lemma

    normalizer = Normalizer()
    lemmatizer = Lemmatizer()
    stemmer = Stemmer()
    tagger = POSTagger(model = 'D:\\pos_tagger.model')


    normal_text=normalizer.normalize(textt)

    sents_text=sent_tokenize(normal_text)
    
    words_text = word_tokenize(normal_text)

    parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)

    f1 = open(stop_word, "r",encoding='utf-8')
    stop_words = f1.read()
    stop_words = stop_words.split("\n")

    words_without_stopwords = []
    for w in words_text:
        if w not in stop_words:
            words_without_stopwords.append(w)


    words_tag = tagger.tag(words_without_stopwords)
    
    fixd_lemma_words=[]
    lemma_words = []
    for w in words_tag:
        if w[1] == 'ADJ':
            lemma_words.append([stemmer.stem(w[0]), w[1]])
        else:
            lemma_words.append([lemmatizer.lemmatize(w[0]), w[1]])
    
    for i in range(len(lemma_words)):
        fixd_lemma_words.append(lemma_words[i][0])
    fixdd_lemma=' '.join(fixd_lemma_words)
    
    return (fixdd_lemma)

def bag_of_words():

    global unic_text
    global bag_of_text

    unic_text=list(set(fixd_lemma_words))
    print('unic_text : ', unic_text)
    print("\n")

    bag_of_text = {}
    for i in fixd_lemma_words:
        word = i.lower()
        if word in bag_of_text:
            bag_of_text[word] += 1
        else:
            bag_of_text[word] = 1

    print ('bag_of_text : ',bag_of_text)

    n_words_set=len(bag_of_text.keys())
    b=list(bag_of_text.values())

    df_tf=pd.DataFrame(np.zeros((1, n_words_set)),columns=bag_of_text.keys())

    i=0
    for w in bag_of_text.keys():
        df_tf[w] = b[i]
        i+=1
    return df_tf


def N_gram(n):

    CountVec = CountVectorizer(ngram_range=(n,n))
    
    
    Count_data = CountVec.fit_transform([fixdd_lemma])

    cv_dataframe=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names_out())
    print (cv_dataframe)
    
    print("\n")
    print("tf_idf Without Smoothing:")
    print("\n")

    tf_idf_vec = TfidfVectorizer(use_idf=True,
                            smooth_idf=False,
                            ngram_range=(n,n))

    tf_idf_data = tf_idf_vec.fit_transform([fixdd_lemma])


    tf_idf_dataframe=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names_out())
    print(tf_idf_dataframe)
    print("\n")

    tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,
                            smooth_idf=True,
                            ngram_range=(n,n))


    tf_idf_data_smooth = tf_idf_vec_smooth.fit_transform([fixdd_lemma])

    print("tf_idf With Smoothing:")
    print("\n")
    tf_idf_dataframe_smooth=pd.DataFrame(tf_idf_data_smooth.toarray(),columns=tf_idf_vec_smooth.get_feature_names_out())
    print (tf_idf_dataframe_smooth)


def tf_idf():

    n_docs = 1
    n_words_set = len(unic_text)

    df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), columns=unic_text)

    for w in fixd_lemma_words:
        df_tf[w] = df_tf[w] + (1 / len(words_text))

    dictt={}
    column=df_tf.columns
    for i in column:
        dictt[i]=df_tf[i][0]

    print(dictt)

    return df_tf



def sentiment(c):

    global emtiyaz

    negetive = open('D://python/proje/sentiment/sentiment-lexicons/negative_words_fa.txt', "r",encoding='utf-8')
    positive = open('D://python/proje/sentiment/sentiment-lexicons/positive_words_fa.txt', "r",encoding='utf-8')

    neg=negetive.read()
    pos=positive.read()

    neg_split=neg.split('\n')
    pos_split=pos.split('\n')

    emtiyaz_kol=0

    emtiyaz=[]

    words_matn=[]

    for i in c:
        words_matn.append(word_tokenize(i))

    for j in words_matn:

        emtiyaz_pos=0
        emtiyaz_neg=0

        for k in neg_split:
            if k in j :
                emtiyaz_neg+=1
        for b in pos_split:
            if b in j :
                emtiyaz_pos+=1

        em=emtiyaz_pos-emtiyaz_neg
        emtiyaz.append(em)

        if em>0:
            print('positive')

        elif em<0:
            print('negitive')

        elif em==0:
            print('khonsa')

    for i in emtiyaz:
        emtiyaz_kol+=i

    print('\n')
    print('natige kol : ')
    
    if emtiyaz_kol>0:
        print('positive')
    elif emtiyaz_kol<0:
        print('negitive')
    elif emtiyaz_kol==0:
        print('konsa')
    


In [15]:
splitt('D://python/proje/matn.txt')

['پس از آغاز سهمیه\u200cبندی بنزین<a target="_blank" class="link__CustomNextLink-sc-1r7l32j-0 iJNEeE LinkElement__LinkBase-sc-1i205bq-1 gizzxK" href="/car/"></a>، تاکسی\u200cهای اینترنتی برای تامین نیاز رانندگان خود با چالش\u200cهای مختلفی رو\u200cبه\u200cرو شدند. با توجه به این سهمیه\u200cبندی، اسنپ چند سالی است برای رانندگانی که ماهانه <strong>۲۰۰ کیلومتر</strong> در سفرهایشان با اسنپ تردد می\u200cکنند سهمیه بنزین اعتباری در نظر گرفته است.\nاگر راننده اسنپ هستید یا قصد دارید به جمع رانندگان اسنپ بپیوندید و دغدغه تأمین بنزین برای خودروی خود را دارید در این مطلب با زومیت همراه باشید تا هم برایتان توضیح دهیم که سهمیه بنزین اعتباری اسنپ چه جزئیاتی دارد و چطور می\u200cتوانید از آن استفاده کنید و هم شما را با نحوه استعلام این سهمیه از سامانه سماس آشنا کنیم.\n<mark style="background-color:#fcd430">این مقاله در تاریخ ۲۷ تیر ۱۴۰۲ با جدیدترین اطلاعات به\u200cروزرسانی شده است.</mark>\nهمان\u200cطور که گفتیم، اسنپ برای این که به دغدغه تأمین بنزین رانندگان خود پاسخ دهد و به آن\u200cها کمک کند تا 

In [16]:
preprocess(text[0],'D:\\Pesian_Stop_Words_List.txt')

'آغاز سهمیه\u200cبندی بنزین<a ۱ r ۷ l ۳۲ j- iJNEeE ۱ i ۲۰۵ ۱ car >< a> تاکسی\u200cهای اینترنت تامین نیاز رانندگان چالش مختلف رو\u200cبه\u200cرو توجه سهمیه\u200cبندی اسنپ سال رانندگانی ماهانه <strong> ۲۰۰ کیلومتر< strong> سفر اسنپ تردد کرد#کن سهمیه بنزین اعتبار نظر گرفت#گیر راننده اسنپ قصد جمع رانندگان اسنپ پیوست#پیوند دغدغه تأمین بنزین خودرو مطلب زوم همراه برایتان توضیح سهمیه بنزین اعتباری اسنپ جزئیات توانست#توان نحوه استعلام سهمیه سامانه سماس آشنا <mark style= #fcd ۴۳۰ >این مقاله تاریخ ۲۷ تیر ۱۴۰۲ جدید اطلاعات به\u200cروزرسان شد#شو < mark> همان\u200cطور اسنپ این\u200cکه دغدغه تأمین بنزین رانندگان پاسخ آن کمک سهمیه سوخت دریافت طرح سهمیه بنزین اعتباری سال اجرا هرچند شرایط میزان سهمیه سال متفاوت سهمیه رانندگانی تعلق گرفت#گیر سرویس\u200cهای اسنپ ماه حداقل ۲۰۰ کیلومتر تردد داشت#دار طبق اسنپ اعتبار سهمیه سوخت نهاد دولت واریز نکته ذکر شد#شو اسنپ اسامی رانندگان واجد شرایط دریافت سهمیه سوخت میزان پیمایش ماهانه آن اختیار نهاد قرار داد#ده اطلاعات راننده ارائه شیوه تخصیص اعتبار سوخت رانندگان اسنپ

In [18]:
bag_of_words()

unic_text :  ['rel=', 'سمت', 'توانست#توان', 'عبور', 'تک\u200cسوز', 'برداشت', 'داد#ده', 'نظر', 'ایجاد', 'مبلغ', '۴', 'موضوع', 'محاسبه', 'کمک', 'ماه', 'کد', '<strong>دستگاه\u200cهای', 'خانواده', 'اجرا', 'اپلیکیشن', 'درصد', 'مسافر', '۱۵۰۰', 'ترتیب', '۴۰', 'آخر', 'کرد#کن', 'نرخ', 'داده\u200cشده', 'خط', 'کلیک', 'فعال', '۱۸۰', 'شناسه', 'تاریخچه', 'تومان', 'کشید#کش', 'رو\u200cبه\u200cرو', 'جدید', 'تخصیص', 'کار', 'style=', 'بانکی\u200cشان', 'حداقل', 'مدل', '۱۰۰', 'مابقی', 'سیستم', 'شد#شو', 'دوم', '۳۰۰', 'ضرب', 'هنگامی\u200cکه', 'سهمیه', 'پرسید#پرس', 'همراه', 'سقف', 'دست', 'تاریخ', 'آزاد', '۳۲', '۲۷', 'این\u200cکه', 'شیوه', 'ثالث', 'سرویس', '۱۷۵', 'مشغول', 'سفر', '<strong>رمز', 'ثبت\u200cنام', 'یاخت#یاز', 'ارائه', 'کیلومتر', 'مستلزم', 'j-', 'دریافت', 'قبول', '۳۰۰۰', '٫', 'سهمیه\u200cبندی', 'مربوطه', 'ملت', 'ابتدا', 'تعیین\u200cشده', 'واریز', 'دوگانه\u200cسوز', 'اسامی', 'مطلب', 'لیتر', 'کارت', 'اعتباری', 'ملی', 'iJNEeE', 'تاکسی\u200cهای', 'سال', 'گرفت#گیر', 'ملی<', 'نحوه', 'تأمین', '۶۰', 'واریزش

,آغاز,سهمیه‌بندی,بنزین<a,۱,r,۷,l,۳۲,j-,ijneee,...,عبور,پرسید#پرس,<strong>رمز,سوخت<,آخر,سایت,چاپی,پیش,باقی‎مانده,خانواده
0,1,2,1,6,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1


In [38]:
N_gram(2)

   car تاکسی  fcd ۴۳۰  https samas  ijneee ۲۰۵  ir سامانه  ir مراجعه  \
0          1        1            1           2          1          1   

   mark style  mark همان  moi ir  rel ۳۲  ...  ۲۶۲ ۵۰۰  ۲۷ تیر  ۳۰۰ لیتر  \
0           1          1       2       1  ...        1       1         2   

   ۳۰۰۰ تومان  ۳۲ ijneee  ۴۰ درصد  ۴۳۰ این  ۵۰۰ تومان  ۶۰ لیتر  ۷۵ لیتر  
0           1          2        1        1          3        1        3  

[1 rows x 854 columns]


tf_idf Without Smoothing:


   car تاکسی   fcd ۴۳۰  https samas  ijneee ۲۰۵  ir سامانه  ir مراجعه  \
0   0.018598  0.018598     0.018598    0.037197   0.018598   0.018598   

   mark style  mark همان    moi ir    rel ۳۲  ...   ۲۶۲ ۵۰۰    ۲۷ تیر  \
0    0.018598   0.018598  0.037197  0.018598  ...  0.018598  0.018598   

   ۳۰۰ لیتر  ۳۰۰۰ تومان  ۳۲ ijneee   ۴۰ درصد   ۴۳۰ این  ۵۰۰ تومان   ۶۰ لیتر  \
0  0.037197    0.018598   0.037197  0.018598  0.018598   0.055795  0.018598   

    ۷۵ لیتر  
0  0.055795  

[1 rows x 854 colu

In [45]:
tf_idf()

{'rel=': 0.0004791566842357451, 'سمت': 0.0009583133684714902, 'توانست#توان': 0.005270723526593196, 'عبور': 0.0004791566842357451, 'تک\u200cسوز': 0.0019166267369429804, 'برداشت': 0.0004791566842357451, 'داد#ده': 0.006229036895064686, 'نظر': 0.004312410158121706, 'ایجاد': 0.0004791566842357451, 'مبلغ': 0.004312410158121706, '۴': 0.0009583133684714902, 'موضوع': 0.0004791566842357451, 'محاسبه': 0.0023957834211787254, 'کمک': 0.0004791566842357451, 'ماه': 0.0014374700527072354, 'کد': 0.0033540967896502154, '<strong>دستگاه\u200cهای': 0.0004791566842357451, 'خانواده': 0.0004791566842357451, 'اجرا': 0.0004791566842357451, 'اپلیکیشن': 0.0023957834211787254, 'درصد': 0.0004791566842357451, 'مسافر': 0.0009583133684714902, '۱۵۰۰': 0.0014374700527072354, 'ترتیب': 0.0004791566842357451, '۴۰': 0.0004791566842357451, 'آخر': 0.0004791566842357451, 'کرد#کن': 0.007187350263536176, 'نرخ': 0.0014374700527072354, 'داده\u200cشده': 0.0004791566842357451, 'خط': 0.0004791566842357451, 'کلیک': 0.001437470052707235

,rel=,سمت,توانست#توان,عبور,تک‌سوز,برداشت,داد#ده,نظر,ایجاد,مبلغ,...,l,لازم,۲۰,اطلاعات,انقضا,متفاوت,عبارت‌اند,جزئیات,نسبت,۷۵
0,0.000479,0.000958,0.005271,0.000479,0.001917,0.000479,0.006229,0.004312,0.000479,0.004312,...,0.000958,0.003833,0.000479,0.00575,0.000479,0.000479,0.000479,0.000479,0.000479,0.001437


In [79]:
a=True
splitt('D://python/proje/matn.txt')

for i in text:
    preprocess(i,'D:\\Pesian_Stop_Words_List.txt')
    p_text.append(fixdd_lemma)

sentiment(p_text)

positive
positive
khonsa
negitive
negitive
positive
positive
khonsa
positive
negitive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
khonsa
positive
positive
positive
khonsa
positive
positive
positive
positive
positive
positive
khonsa
negitive
negitive
positive
positive
khonsa
positive
negitive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
khonsa
positive
positive
positive
khonsa
positive
positive
positive
positive
positive
positive
khonsa
negitive
negitive
positive
positive
khonsa
positive
negitive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
positive
khonsa
positive
positive
positive
khonsa
positive
positive
positive
positive
positive
positive
khonsa
negitive
negitive
positive
positive
khonsa
positive
negitive
positive
positive
positive
positive
positive
positive
positive
positive
po